In [1]:
import os
import json
import dateparser
import datetime
from ladder_page import LadderPage
from history_page import HistoryPage
from retry_decorator import retry

In [14]:
@retry(Exception, tries=5, timeout_secs=10)
def get_new_games(player, server, last_bq_date=None):
    start_date_for_db = datetime.datetime(2018, 1, 1)
    def no_more_dates():
        if last_bq_date is None:
            return False
        else:
            return current_date <= last_bq_date
        
    def no_more_pages():
        return page >= next_page
    
    new_games = []
    page = 1
    
    while True:
#         print(page)
        history_page = HistoryPage(player, server, page)
        games = history_page.games
        next_page = history_page.next_page

        for d in games:
            current_date = dateparser.parse(d.get('date'))
            if no_more_dates() or (current_date < start_date_for_db):
                return new_games
            else:
                new_games.append(d)

        if no_more_pages():
            return new_games
        else:
            page = next_page

In [ ]:
servers = ['azeroth', 'northrend']
for server in servers:
    dir_ = './data_backfill/partial/{}'.format(server)
    if not os.path.exists(dir_):
        os.makedirs(dir_)
    
    page = 1
    # Loop over the first 40 ladder pages.
    for page in range(1, 40):
        ladder_page = LadderPage(server, page=page)
        print(page)
        
        # On each page select a player and get their games.
        for row in ladder_page.rows:
            games = []
            player = row.get('player')
            print(player)
            
            file_path = '{}/{}.json'.format(dir_, player)
            if os.path.exists(file_path):
                print('{} - Already exists'.format(player))
                continue

            games = get_new_games(player, server)
            print('{} - {}'.format(player, len(games)))
            with open(file_path, 'w') as f:
                json.dump(games, f)
            print('written')

1
Pieck
Pieck - Already exists
Cocaine.
Cocaine. - Already exists
Ceron
Ceron - Already exists
Ludique
Ludique - Already exists
cutegirl90
cutegirl90 - Already exists
^Divine
^Divine - Already exists
niGhT-BlOoDy
niGhT-BlOoDy - Already exists
lRain
lRain - Already exists
Lado
Lado - Already exists
STLER
STLER - Already exists
Aw]DanGer
Aw]DanGer - Already exists
SwordPuppyTV
SwordPuppyTV - Already exists
girls-love-me
girls-love-me - Already exists
HumanKingTomViP
HumanKingTomViP - Already exists
Human-in-1.30
Human-in-1.30 - Already exists
sayso.
sayso. - Already exists
InFs]Psycho
InFs]Psycho - Already exists
HOT.PriesT
HOT.PriesT - Already exists
Che
Che - Already exists
DEScn
DEScn - Already exists
2
INHELL.
INHELL. - Already exists
Mr.Satan
Mr.Satan - Already exists
Alf
Alf - Already exists
JusTforFun
JusTforFun - Already exists
Anewb.
Anewb. - 116
written
ANGRY_KOREA_MAN


ERROR:root:Retrying in 10.92 seconds ...
Traceback (most recent call last):
  File "/anaconda/lib/python3.6/site-packages/urllib3/connectionpool.py", line 387, in _make_request
    six.raise_from(e, None)
  File "<string>", line 2, in raise_from
  File "/anaconda/lib/python3.6/site-packages/urllib3/connectionpool.py", line 383, in _make_request
    httplib_response = conn.getresponse()
  File "/anaconda/lib/python3.6/http/client.py", line 1331, in getresponse
    response.begin()
  File "/anaconda/lib/python3.6/http/client.py", line 297, in begin
    version, status, reason = self._read_status()
  File "/anaconda/lib/python3.6/http/client.py", line 258, in _read_status
    line = str(self.fp.readline(_MAXLINE + 1), "iso-8859-1")
  File "/anaconda/lib/python3.6/socket.py", line 586, in readinto
    return self._sock.recv_into(b)
socket.timeout: timed out

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/anaconda/lib/python3

In [ ]:
def wash_player_name(player):
    return player.replace('|c', '%7')

In [ ]:
import requests
from bs4 import BeautifulSoup
from helpers import wash_player_name
from config import data_positions_history
from urllib.parse import parse_qs
import pandas as pd
import json


class HistoryPage:
    def __init__(self, player, server, page=1):
        self.player = player
        self.server = server.title()
        self.page = page
        self.params = {'PlayerName': self.player, 'Gateway': self.server, 'PageNo': page}
        self.url = 'http://classic.battle.net/war3/ladder/w3xp-player-logged-games.aspx'
        self.servers = ['azeroth', 'lordaeron', 'northrend', 'kalimdor']
        self.soup = self.get_soup()
        self._validate()

    def __str__(self):
        return '{}@{}'.format(self.player, self.server)

    def __repr__(self):
        return '{}@{}'.format(self.player, self.server)

    def _validate(self):
        self.validate_server()
        self.validate_player()

    def validate_player(self):
        error_span = self.soup.find('span', class_='colorRed')
        if error_span is not None:
            raise Exception('{} | Profile not found'.format(str(self)))

    def validate_server(self):
        if self.server.lower() not in self.servers:
            raise Exception('{} | Invalid server'.format(str(self)))

    def get_soup(self):
        try:
            r = requests.get(self.url, params=self.params)
        except requests.exceptions.RequestException as e:
            print(e)

        return BeautifulSoup(r.content, 'lxml')

    @property
    def game_containers(self):
        return self.soup.find('table', id='tblGames').find_all('tr', class_='rankingRow')[1:]

    def games(self):
        for game in self.game_containers:
            yield Game(self.player, game).parse()

    @property
    def next_page(self):
        url = self.soup.find(text='Next\xa0Page').parent.get('href')
        if url is not None:
            next_page = parse_qs(url).get('PageNo')[0]
            return int(next_page)


class Game:
    def __init__(self, player, soup):
        self.player = player
        self.soup = soup

    def parse(self):
        values = self.soup.find_all(['td'])
        values = [x.get_text().strip() for x in values]
        data = self.parse_values(values)
        game_id = self.soup.find_all(['td'])[0].a.get('href').split('&GameID=')[1]
        data['game_id'] = int(game_id)
        data['team_one'].append(self.player)

        for players in ['team_one', 'team_two']:
            data[players] = [wash_player_name(player) for player in data[players]]

        if data['winner'] == 'Win':
            data['winner'] = data['team_one']
        else:
            data['winner'] = data['team_two']

        return data

    @staticmethod
    def parse_values(values):
        data = {}
        for field, meta_data in data_positions_history.items():
            i = meta_data['position']
            v = values[i]
            formatter = meta_data['function']
            if formatter:
                value = formatter(v)
            data[field] = value

        return data

if __name__ == '__main__':
    players = [
        {
            'player': 'Rellik',
            'server': 'northrend'
        }
    ]

    print('-- Testing --')
    for player in players:
        page = 1
        data_all = []

        while True:
                history_page = HistoryPage(player.get('player'), player.get('server'), page)
                data = list(history_page.games())
                data_all.extend(data)
                next_page = history_page.next_page
                if page >= next_page:
                        break
                page = next_page

        df = pd.DataFrame(data_all)
        print(df.shape)
        print(df.head(3))

    data = df.to_dict(orient='records')
    print(data)
    with open('./data_backfill/{}.json'.format(player.get('player')), 'w') as f:
        json.dump(data, f)


In [ ]:
class Game:
    def __init__(self, player, soup):
        self.player = player
        self.soup = soup
        
    
    def parse(self):
        values = self.soup.find_all(['td'])
        values = [x.get_text().strip() for x in values]
        data = self.parse_values(values)
        id_ = self.soup.find_all(['td'])[0].a.get('href').split('&GameID=')[1]
        data['game_id'] = int(id_)
        data['team_one'].append(self.player)
        
        for players in ['team_one', 'team_two']:
            data[players] = [wash_player_name(player) for player in data[players]]
            
        if data['winner'] == 'Win':
            data['winner'] = data['team_one']
        else:
            data['winner'] = data['team_two']
            
        
        return data
     
    
    @staticmethod
    def parse_values(values):
        data = {}
        for field, meta_data in data_positions_history.items():   
            i = meta_data['position']
            v = values[i]
            formatter = meta_data['function']
            if formatter:
                value = formatter(v)
            data[field] = value

        return data

In [ ]:
history_page = HistoryPage('followgrubby', 'northrend')
print(history_page)

In [ ]:
data = list(history_page.games())
df = pd.DataFrame(data)
df

In [ ]:
page = 1
data_all = []

while True:
    history_page = HistoryPage('followgrubby', 'northrend', page)
    data = list(history_page.games())
    data_all.extend(data)
    next_page = history_page.next_page

    if page >= next_page:
        break
    page = next_page


In [ ]:
df = pd.DataFrame(data_all)
df

In [ ]:
df.columns

In [ ]:
"""Scrapes a WC3 profile page from battlenet and returns json."""
from bs4 import BeautifulSoup
from botocore.vendored import requests
import dateparser
import pandas as pd
from config import data_positions


class Profile:
    def __init__(self, player, server):
        self.player = player
        self.server = server.title()
        self.params = {'PlayerName': self.player, 'Gateway': self.server}
        self.url = 'http://classic.battle.net/war3/ladder/w3xp-player-profile.aspx?'
        self.servers = ['azeroth', 'lordaeron', 'northrend', 'kalimdor']
        self.soup = self.get_soup()
        self._validate()
        self.tables = self._parse_tables()

    def __str__(self):
        return '{}@{}'.format(self.player, self.server)

    def get_soup(self):
        try:
            r = requests.get(self.url, params=self.params)
        except requests.exceptions.RequestException as e:
            print(e)

        return BeautifulSoup(r.content, 'lxml')

    def _validate(self):
        self.validate_server()
        self.validate_player()

    def parse(self):
        data = {}
        data['info'] = self.information
        data['individual'] = self.individual
        data['team'] = self.team
        return data

    def _parse_tables(self):
        soup = self.soup.find('table', class_='mainTable')
        tables = soup.find_all('td', {'align': 'center', 'valign': 'top'})
        tables = dict(zip(['info', 'individual', 'team'], tables))
        return tables

    @property
    def information(self):
        return {
            'player': self.player,
            'server': self.server,
            'clan': self.clan,
            'main_race': self.main_race,
            'home_page': self.home_page,
            'additional_info': self.parse_additional_info,
            'last_ladder_game': self.last_ladder_game
        }

    @property
    def individual(self):
        soup = self.tables.get('individual')
        type_ = 'individual'
        vocab = {'Team Games': 'random_team', 'Solo Games': 'solo', 'FFA Games': 'free_for_all'}
        data = {}

        for game_type, new_key in vocab.items():
            container = soup.find(text=game_type)
            if container is not None:
                table = container.parent.parent.parent.parent.parent
                values = [x.get_text() for x in table.find_all('b')]
                d = self.format_values(type_, values)
                d['win_percentage'] = self.calc_win_percentage(d['wins'], d['losses'])
                new_key = vocab[game_type]
                data[new_key] = d

        return data

    @property
    def team(self):
        soup = self.tables.get('team')
        type_ = 'teams'
        teams = soup.find_all(text='Partner(s):')
        data = []

        for team in teams:
            table = team.parent.parent.parent.parent.parent.parent.parent.parent
            values = self.extract_values(table)
            d = self.format_values(type_, values)
            d['win_percentage'] = self.calc_win_percentage(d['wins'], d['losses'])
            data.append(d)

        return data

    @property
    def home_page(self):
        soup = self.tables.get('info')
        home_page = soup.find('div', {'id': 'homePage'})
        home_page = home_page.b.get_text().strip()
        if home_page == '':
            return 'N/A'
        return home_page

    @property
    def parse_additional_info(self):
        soup = self.tables.get('info')
        additional_info_div = soup.find('div', {'id': 'additionalInfo'})
        script = additional_info_div.script.get_text()
        text_start = 'document.write("'
        i = script.find(text_start)
        if i == -1:
            additional_info = None
        else:
            i += len(text_start)

        text_end = '");'
        j = script[i:].find(text_end)
        additional_info = script[i: i + j]

        if additional_info in ['', None]:
            return 'N/A'
        return additional_info

    @property
    def last_ladder_game(self):
        soup = self.tables.get('info')
        last_ladder_game = soup.find(text='Last Ladder Game:')
        if last_ladder_game is not None:
            last_ladder_game = last_ladder_game.parent.parent.b.get_text()
        last_ladder_game = str(dateparser.parse(last_ladder_game).strftime('%Y-%m-%d'))
        return last_ladder_game

    @property
    def main_race(self):
        soup = self.tables.get('info')
        overall_stats_table = soup.find('td', class_='rankingHeader')
        if overall_stats_table is not None:
            overall_stats_table = overall_stats_table.parent.parent
        rows = overall_stats_table.find_all('tr')[1:-1]
        df = self.parse_stats_table(rows)
        if len(df[df['percentage_games'] >= 75]) != 0:
            main_race = df[df['percentage_games'] >= 75]['race'].values[0].title()
        else:
            main_race = 'No main race'

        return main_race

    @staticmethod
    def parse_stats_table(rows):
        data = []
        keys = ['race', 'wins', 'losses', 'win_percentage']

        for row in rows:
            row = [x.get_text().strip() for x in row.find_all('td')]
            row = dict(zip(keys, row))
            row['num_games'] = int(row['losses']) + int(row['wins'])
            data.append(row)

        df = pd.DataFrame(data)
        df['percentage_games'] = df['num_games'] * 100 / sum(df['num_games'])
        df['race'] = df['race'].apply(lambda x: x.lower().replace(':', ''))

        for col in ['wins', 'losses']:
            df[col] = df[col].astype(int)

        df['total_games'] = df['losses'] + df['wins']

        return df

    @property
    def clan(self):
        soup = self.tables.get('info')
        clan_url = soup.find('a', href=lambda x: x and x.startswith('w3xp-clan-profile.aspx?'))
        if clan_url is not None:
            return clan_url.get_text()
        return 'N/A'

    @staticmethod
    def format_values(type_, values):
        fields = ['wins', 'losses', 'partners', 'level', 'rank', 'experience']
        data = {}

        for field in fields:
            meta_data = data_positions[type_][field]
            if not meta_data:
                continue
            i = meta_data['position']
            v = values[i]
            if not v:
                continue
            formatter = meta_data['function']
            if formatter:
                value = formatter(v)
            else:
                value = v

            data[field] = value

        return data

    @staticmethod
    def calc_win_percentage(wins, losses):
        win_percentage = round((100 * int(wins)) / (int(wins) + int(losses)), 2)
        return '{0:.2f}%'.format(win_percentage)

    @staticmethod
    def extract_values(table):
        values = []
        values_old = table.find_all('b')

        for i, value in enumerate(values_old):
            if i == 3:
                partners = [x.get_text() for x in value]
                if len(partners) > 1:
                    partners.remove('')
                values.append(partners)
            else:
                values.append(value.get_text())

        return values

    def validate_player(self):
        error_span = self.soup.find('span', class_='colorRed')
        if error_span is not None:
            raise Exception('{} | Profile not found'.format(str(self)))

    def validate_server(self):
        if self.server.lower() not in self.servers:
            raise Exception('{} | Invalid server'.format(str(self)))

    def request_solo(self):
        data = self.individual.get('solo')
        if data is not None:
            data.update({
                'main_race': self.main_race,
                'player': self.player,
                'server': self.server,
            })
        return data

    def request_random_team(self):
        data = self.individual.get('random_team')
        if data is not None:
            data.update({
                'main_race': self.main_race,
                'player': self.player,
                'server': self.server,
            })
        return data

    def request_info(self):
        data = self.information
        if data is not None:
            return data

if __name__ == '__main__':
    players = [
        {
            'player': 'romantichuman',
            'server': 'northrend'
        },
        {
            'player': 'pieck',
            'server': 'northrend'
        },
        {
            'player': 'pieck',
            'server': 'azeroth'
        },
        {
            'player': 'wearefoals',
            'server': 'northrend'
        },
        {
            'player': 'wearefoals',
            'server': 'azeroth'
        },
        {
            'player': 'wearefoals',
            'server': 'fake_server'
        },
        {
            'player': 'fake_player',
            'server': 'azeroth'
        },
        {
            'player': 'moon1234',
            'server': 'northrend'
        },
    ]
    print('-- Testing --')
    for player in players:
        print()
        try:
            profile = Profile(**player)
            print(profile)
            print(profile.parse())
        except Exception as e:
            print(e)
